In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Fits to Data With $J=1,3$ Waves
These fits are similar to our final waveset, but add a set of $J=3$ waves to attempt to extract a potential $\rho_3(1690)$ shape. 

Fit Details:
* Spin 1 and 3 waves, with both reflectivities and spin-projections $|m|<3$
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D, 3^-F$}
* No isotropic background
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-3/no-iso-unconstrained/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
# t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"] #TODO: left out until last 2 bins done
t_bins = ["t_0.10-0.16", "t_0.16-0.23"] 
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]
NUM_T_BINS = len(binned_results)
FIG_WIDTH = 5 * NUM_T_BINS

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Analysis

### Standard Plots
These plots are always handy to have

In [ ]:
fig, axs = plt.subplots(1, NUM_T_BINS, figsize=(15, 6))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/waves_fractional.pdf")  
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/matrix.pdf") 
    plt.show()

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.04))

fig, axs = plt.subplots(
    len(sig_moments), NUM_T_BINS, figsize=(FIG_WIDTH, 25), layout="constrained", sharex=True, sharey="row"
)

row_max = {}
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        label = utils.convert_moment_name(moment) if col == 0 else ""
        my_kwargs = {moment: {"label": label}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)
        if col != 0:
            ax.set_ylabel("")
        max_val = ax.get_ylim()[1]
        print(moment)
        print(max_val)
        if row not in row_max or max_val > row_max[row]:
            row_max[row] = max_val
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")


for col, (t_bin, results) in enumerate(binned_results.items()):
    for row, moment in enumerate(sig_moments):
        axs[row, col].set_ylim(top=row_max[row])
    

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")


### D/S Ratio and Phase

In [ ]:
fig, axs = plt.subplots(
    3,
    NUM_T_BINS,    
    sharey="row",
    figsize=(FIG_WIDTH, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "p1ppS", "m1ppS", "m1p0D"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()
plt.savefig(f"{STUDY_DIR}/ds_ratio_all_t_bins.pdf")

In [ ]:
fig, axs = plt.subplots(
    3,
    NUM_T_BINS,    
    sharey="row",
    figsize=(FIG_WIDTH, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    NUM_T_BINS,    
    sharey="row",
    figsize=(FIG_WIDTH, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves= ["m1pmS", "p1ppS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    NUM_T_BINS,    
    sharey="row",
    figsize=(FIG_WIDTH, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "p1ppS", "p1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()


### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    NUM_T_BINS,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(15, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()
    

max_y=0
for i in range(NUM_T_BINS):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(NUM_T_BINS):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

### $F_m^{(+)}$ and $1^{-}P_1^{(+)}$ Interference
Note that the P-wave is chosen to be the "reference" wave for these fits as well, as its strongest over the mass spectrum

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

natural_f_waves = ["p3mnF", "p3mmF", "p3m0F", "p3mpF", "p3mqF"]

for f_wave in natural_f_waves:

    fig, axs = plt.subplots(
        2,
        NUM_T_BINS,
        sharex=True,        
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1],
        figsize=(20, 10),
        layout="constrained",
    )
    for i, (t_bin, results)in enumerate(binned_results.items()):
        colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

        results.plot.phase.mass_phase(
            amp1="p1mpP", amp2=f_wave,
            amp1_kwargs={"color":colors[3], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=axs[0, i],
            phase_ax=axs[1, i],
        )
        t_low, t_high = results.get_t_edges()
        axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if i != 0:
            axs[0,i].set_ylabel("")
            axs[1,i].set_ylabel("")
            axs[0,i].get_legend().remove()

        ax_inset = inset_axes(
            axs[0, i],
            width="30%",
            height="40%",
            loc="center right",
            bbox_to_anchor=(0, 0.1, 1, 1),
            bbox_transform=axs[0, i].transAxes,
            borderpad=2,
        )
        results.plot.phase.mass_phase(
            amp1="p1mpP", amp2=f_wave,
            amp1_kwargs={"color":colors[3], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=ax_inset,
            phase_ax=axs[1,i],
        )
        ax_inset.set_xlim(1.50, 1.80)

        # zoom in on region 
        indices = results.get_fit_indices(1.50, 1.80)    
        f_wave_inset_values = results.fit_df.loc[indices, f_wave]
        ax_inset.set_ylim(
            f_wave_inset_values.min() * 0.8,
            f_wave_inset_values.max() * 1.2,
        )    

        # add lines connecting inset to main plot
        mark_inset(
            axs[0, i],
            ax_inset,
            loc1=3,
            loc2=4,
            fc="none",
            ec="0.5",
            lw=1.0,
            alpha=0.75,
        )

        ax_inset.legend().remove()
        ax_inset.set_ylabel("")
        ax_inset.tick_params(labelsize=12)
        ax_inset.set_title("")  
        
        xticks = np.arange(1.50, 1.80 + 1e-9, 0.05)
        ax_inset.set_xticks(xticks)
        ax_inset.set_xticklabels([f"{x:.2f}" for x in xticks])

    max_y=0
    for i in range(NUM_T_BINS):
        y_max = axs[0,i].get_ylim()[1]
        max_y = max(max_y, y_max)
    for i in range(NUM_T_BINS):
        axs[0,i].set_ylim(0, max_y)

    plt.show()


# plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

Try the unnatural exchange, for fun

In [ ]:
natural_f_waves = ["m3mnF", "m3mmF", "m3m0F", "m3mpF", "m3mqF"]

for f_wave in natural_f_waves:

    fig, axs = plt.subplots(
        2,
        NUM_T_BINS,
        sharex=True,        
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1],
        figsize=(20, 10),
        layout="constrained",
    )
    for i, (t_bin, results)in enumerate(binned_results.items()):
        colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

        results.plot.phase.mass_phase(
            amp1="m1mpP", amp2=f_wave,
            amp1_kwargs={"color":colors[3], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=axs[0, i],
            phase_ax=axs[1, i],
        )
        t_low, t_high = results.get_t_edges()
        axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if i != 0:
            axs[0,i].set_ylabel("")
            axs[1,i].set_ylabel("")
            axs[0,i].get_legend().remove()

        ax_inset = inset_axes(
            axs[0, i],
            width="30%",
            height="40%",
            loc="center right",
            bbox_to_anchor=(0, 0.1, 1, 1),
            bbox_transform=axs[0, i].transAxes,
            borderpad=2,
        )
        results.plot.phase.mass_phase(
            amp1="m1mpP", amp2=f_wave,
            amp1_kwargs={"color":colors[3], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=ax_inset,
            phase_ax=axs[1,i],
        )
        ax_inset.set_xlim(1.50, 1.80)

        # zoom in on region 
        indices = results.get_fit_indices(1.50, 1.80)    
        f_wave_inset_values = results.fit_df.loc[indices, f_wave]
        ax_inset.set_ylim(
            f_wave_inset_values.min() * 0.8,
            f_wave_inset_values.max() * 1.2,
        )    

        # add lines connecting inset to main plot
        mark_inset(
            axs[0, i],
            ax_inset,
            loc1=3,
            loc2=4,
            fc="none",
            ec="0.5",
            lw=1.0,
            alpha=0.75,
        )

        ax_inset.legend().remove()
        ax_inset.set_ylabel("")
        ax_inset.tick_params(labelsize=12)
        ax_inset.set_title("")  
        
        xticks = np.arange(1.50, 1.80 + 1e-9, 0.04)
        ax_inset.set_xticks(xticks)
        ax_inset.set_xticklabels([f"{x:.2f}" for x in xticks])

    max_y=0
    for i in range(NUM_T_BINS):
        y_max = axs[0,i].get_ylim()[1]
        max_y = max(max_y, y_max)
    for i in range(NUM_T_BINS):
        axs[0,i].set_ylim(0, max_y)

    plt.show()

### $F_m^{(+)}$ and $1^{-}S_0^{(+)}$ Interference
Far enough away from b1, maybe phase motion will be cleaner?

In [ ]:
natural_f_waves = ["p3mnF", "p3mmF", "p3m0F", "p3mpF", "p3mqF"]

for f_wave in natural_f_waves:

    fig, axs = plt.subplots(
        2,
        NUM_T_BINS,
        sharex=True,        
        gridspec_kw={"wspace": 0.0, "hspace": 0.07},
        height_ratios=[3, 1],
        figsize=(20, 10),
        layout="constrained",
    )
    for i, (t_bin, results)in enumerate(binned_results.items()):
        colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

        results.plot.phase.mass_phase(
            amp1="p1p0S", amp2=f_wave,
            amp1_kwargs={"color":colors[2], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=axs[0, i],
            phase_ax=axs[1, i],
        )
        t_low, t_high = results.get_t_edges()
        axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
        if i != 0:
            axs[0,i].set_ylabel("")
            axs[1,i].set_ylabel("")
            axs[0,i].get_legend().remove()

        ax_inset = inset_axes(
            axs[0, i],
            width="30%",
            height="40%",
            loc="center right",
            bbox_to_anchor=(0, 0.1, 1, 1),
            bbox_transform=axs[0, i].transAxes,
            borderpad=2,
        )
        results.plot.phase.mass_phase(
            amp1="p1p0S", amp2=f_wave,
            amp1_kwargs={"color":colors[2], "alpha": 1.0},
            amp2_kwargs={"color":colors[7], "alpha": 1.0},
            amp_ax=ax_inset,
            phase_ax=axs[1,i],
        )
        ax_inset.set_xlim(1.50, 1.80)

        # zoom in on region 
        indices = results.get_fit_indices(1.50, 1.80)    
        f_wave_inset_values = results.fit_df.loc[indices, f_wave]
        ax_inset.set_ylim(
            f_wave_inset_values.min() * 0.8,
            f_wave_inset_values.max() * 1.2,
        )    

        # add lines connecting inset to main plot
        mark_inset(
            axs[0, i],
            ax_inset,
            loc1=3,
            loc2=4,
            fc="none",
            ec="0.5",
            lw=1.0,
            alpha=0.75,
        )

        ax_inset.legend().remove()
        ax_inset.set_ylabel("")
        ax_inset.tick_params(labelsize=12)
        ax_inset.set_title("")  
        
        xticks = np.arange(1.50, 1.80 + 1e-9, 0.04)
        ax_inset.set_xticks(xticks)
        ax_inset.set_xticklabels([f"{x:.2f}" for x in xticks])

    max_y=0
    for i in range(NUM_T_BINS):
        y_max = axs[0,i].get_ylim()[1]
        max_y = max(max_y, y_max)
    for i in range(NUM_T_BINS):
        axs[0,i].set_ylim(0, max_y)

    plt.show()

### Naturalities

In [ ]:
fig, axs = plt.subplots(
    6, NUM_T_BINS, figsize=(FIG_WIDTH, 15), layout="constrained", sharex=True, sharey="row"
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_values = ["p1p", "p1m", "p3m", "m1p", "m1m", "m3m"] 
    labels = ["Natural $1^+$", "Natural $1^-$", "Natural $3^-$", "Unnatural $1^+$", "Unnatural $1^-$", "Unnatural $3^-$"]
    colors = ["tab:red", "tab:red", "tab:red", "tab:blue", "tab:blue", "tab:blue"]
    for row in range(6):
        ax = axs[row, col]    
        label = labels[row] if col == 0 else ""
        val = row_values[row]
        my_kwargs = {val: {"label": label, "color": colors[row]}}
        results.plot.intensity.plot([val], ax=ax, col_kwargs=my_kwargs)

        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)
        if col != 0:
            ax.set_ylabel("")
            if ax.get_legend() is not None:
                ax.get_legend().remove()

plt.savefig(f"{STUDY_DIR}/naturalities_all_t_bins.pdf")

## Comparison to $J=1$ Model

In [ ]:
t_bins = ["t_0.10-0.16", "t_0.16-0.23"] 
spin1_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/../../spin-1/no-iso-unconstrained/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        spin1_results[t_bin] = ResultManager(**data)

spin1_results: dict[str, ResultManager]

### Interference

In [ ]:
fig, axs = plt.subplots(
    2,
    2,
    sharex=True,
    figsize=(15, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    layout="constrained",
)

for i, t_bin in enumerate(t_bins):
    binned_results[t_bin].plot.phase.mass_phase(
        "p1p0S", "p1mpP",
        amp1_kwargs={"color": "black", "markeredgecolor":"black", "marker":"o", "markersize":7, "markerfacecolor":"white", "label":f"(J=1+3) {utils.convert_amp_name("p1p0S")}", "alpha": 0.7},
        amp2_kwargs={"color": "black", "markeredgecolor":"black", "marker":"s", "markersize":7, "markerfacecolor":"white", "label":f"(J=1+3) {utils.convert_amp_name("p1mpP")}", "alpha": 0.7},
        phase_kwargs={"color":"black", "markeredgecolor":"black", "marker":"o", "markersize":3, "markerfacecolor":"white", "label":f"(J=1+3)", "alpha": 0.7},
        amp_ax=axs[0,i],
        phase_ax=axs[1,i],
    )

    spin1_results[t_bin].plot.phase.mass_phase(
        "p1p0S", "p1mpP",
        amp1_kwargs={"color": "green", "marker":"o", "markersize":7, "label":f"(J=1) {utils.convert_amp_name("p1p0S")}", "alpha": 0.7},
        amp2_kwargs={"color": "green", "marker":"s", "markersize":7, "label":f"(J=1) {utils.convert_amp_name("p1mpP")}", "alpha": 0.7},
        phase_kwargs={"color":"green", "marker":"o", "markersize":3, "label":"(J=1)", "alpha": 0.7},
        amp_ax=axs[0,i],
        phase_ax=axs[1,i],
    )

    t_low, t_high = binned_results[t_bin].get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/interference_comparison.pdf", bbox_inches='tight')